# Open and visualise modis l3 data

* you can get the files from https://oceancolor.gsfc.nasa.gov/l3/
* select what you want
* klick download
* use links for download (e.g. put in file browser or use a python script)
* put in datasets dir

## imports

In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

# for geopandas
import geopandas as gpd

# we imported all our things :)

# possible modes: 
modes = [
        { # 0
            "type": "chlor_a", 
            "filename": "../../datasets/modis/AQUA_MODIS.20230101_20230131.L3m.MO.CHL.chlor_a.4km.nc",
            "description": "chlorophyll"
        },
        { # 1
            "type": "sst",
            "filename": "../../datasets/modis/AQUA_MODIS.20230101_20230131.L3m.MO.SST.sst.4km.nc",
            "description": "sea surface temperature"
        },
        { # 2
            "type": "poc",
            "filename": "../../datasets/modis/AQUA_MODIS.20230101.L3m.DAY.POC.poc.4km.nc",
            "description": "Particulate Organic Carbon"
        }
]

mode = modes[1]

## read data

In [ ]:
# read the data
data = nc.Dataset(mode["filename"], "r")

## modify

In [ ]:
print("=== GLOBAL ATTRIBUTES ===")
for attr in data.ncattrs():
    print(f"{attr}: {data.getncattr(attr)}")

print("\n=== DIMENSIONS ===")
for dim, dim_info in data.dimensions.items():
    print(f"{dim}: {len(dim_info)}")

print("\n=== VARIABLES ===")
for var, var_info in data.variables.items():
    print(f"{var}: {var_info}")

print("\n=== GROUPS ===")
for group in data.groups:
    print(group)

# read latitude, longitude and poc data
latitude = data.variables['lat'][:]
longitude = data.variables['lon'][:]
poc_data = data.variables[mode["type"]][:]

# replace fill values with NaN for better plotting
fill_value = data.variables[mode["type"]]._FillValue
poc_data = np.where(poc_data == fill_value, np.nan, poc_data)

# # apply the scale_factor and add_offset
# scale_factor = data.variables['chlor_a'].scale_factor
# add_offset = data.variables['chlor_a'].add_offset
# poc_data = poc_data * scale_factor + add_offset

## crop by coordinates (doesn't work for christina)

In [ ]:
# read and plot the GeoJSON file
geojson_file = 'map22.geojson'  # replace with your GeoJSON file path
gdf = gpd.read_file(geojson_file)

for polygon in gdf.geometry:
    x,y = polygon.exterior.xy
    plt.plot(x, y, color="red")  # adjust the color as you like

# to zoom into the region covered by the GeoJSON file
x_min, y_min, x_max, y_max = gdf.total_bounds

# set any data below 0 to np.nan
my_poc_data = np.where(poc_data < 0, np.nan, poc_data)

# print the max and min values of my_poc_data
print(np.nanmax(my_poc_data))
print('----')
print(np.nanmin(my_poc_data))

In [ ]:
# standard values https://oceancolor.gsfc.nasa.gov/l3/:
  # POC: 10 - 1000 mg m^-3
  # SST: -10 - 50 C
  # chlor_a: 0.01 - 20 mg m^-3

# for each mode, crop out data outside of these ranges (which are land/sea not relevant)
if mode["type"] == "poc":
    my_poc_data = np.where(my_poc_data < 10, np.nan, my_poc_data)
    my_poc_data = np.where(my_poc_data > 1000, np.nan, my_poc_data)
elif mode["type"] == "sst":
    my_poc_data = np.where(my_poc_data < -10, np.nan, my_poc_data)
    my_poc_data = np.where(my_poc_data > 50, np.nan, my_poc_data)
elif mode["type"] == "chlor_a":
    my_poc_data = np.where(my_poc_data < 0.01, np.nan, my_poc_data)
    my_poc_data = np.where(my_poc_data > 20, np.nan, my_poc_data)

# visualisation

## chlorophyll

### save mapped image (chlor_a)

In [ ]:
if mode["type"] == "chlor_a":
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(longitude, latitude, poc_data)
    plt.colorbar(label='Chlorophyll-a (mg m^-3)')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')
    plt.title('MODISA Level-3 Standard Mapped Image for Chlorophyll-a')
    plt.savefig('tmp/modis_chl.png')
    plt.close()

### save histogram (chlor_a)

In [ ]:
if mode["type"] == "chlor_a":
    # plot a histogram of the data excluding values below 1000
    plt.figure(figsize=(12, 6))
    plt.hist(poc_data[~np.isnan(poc_data)], bins=100, range=(0, 87))
    plt.xlabel('Chlorophyll-a (mg m^-3)')
    plt.ylabel('Frequency')
    plt.title('Histogram of chlorophyll-a data')
    plt.savefig('tmp/modis_chl_hist.png')
    plt.close()

### saved cropped image (chlor_a)

In [ ]:
if mode["type"] == "chlor_a":

    # set min and max value
    min_value = 0
    max_value = 20

    # plot data
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(longitude, latitude, poc_data)
    plt.colorbar(label='Chlorophyll-a (mg m^-3)')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')
    plt.title('MODISA Level-3 Standard Mapped Image for Chlorophyll-a')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.clim(min_value, max_value)
    plt.savefig('tmp/modis_chl_crop.png')
    plt.close()

## sea surface temperature

### save mapped image (sst)

In [ ]:
if mode["type"] == "sst":
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(longitude, latitude, poc_data)
    plt.colorbar(label='Temperature')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')
    plt.title('MODISA Level-3 Standard Mapped Image for SST')
    plt.savefig('tmp/modis_sst.png')
    plt.close()

### save histogram (sst)

In [ ]:
if mode["type"] == "sst":
    # todo - does the range make sense??
    # plot a histogram of the data excluding values below 1000
    plt.figure(figsize=(12, 6))
    plt.hist(poc_data[~np.isnan(poc_data)], bins=100, range=(0, 87))
    plt.xlabel('')
    plt.ylabel('')
    plt.title('Histogram of temperature data')
    plt.savefig('tmp/modis_sst_hist.png')
    plt.close()

### save cropped mapped image (sst)

In [ ]:
if mode["type"] == "sst":
    # set min and max value
    min_value = 0
    max_value = 20

    # plot data
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(longitude, latitude, poc_data)
    plt.colorbar(label='Sea Surface Temperature')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')
    plt.title('MODISA Level-3 Standard Mapped Image for SST')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.clim(min_value, max_value)
    plt.savefig('tmp/modis_sst_crop.png')
    plt.close()

## particulate organic carbon

### save mapped image (poc)

In [ ]:
if mode["type"] == "poc":

    # apply the scale_factor and add_offset
    scale_factor = data.variables['poc'].scale_factor
    add_offset = data.variables['poc'].add_offset
    poc_data = poc_data * scale_factor + add_offset

    # plot data
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(longitude, latitude, poc_data)
    plt.colorbar(label='Particulate Organic Carbon (mg m^-3)')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')
    plt.title('MODISA Level-3 Standard Mapped Image for POC')
    plt.savefig('modis_poc.png')
    plt.close()

### save cropped image (poc)

In [ ]:
if mode["type"] == "poc":

    # plot data
    plt.figure(figsize=(12, 6))
    plt.pcolormesh(longitude, latitude, poc_data)
    plt.colorbar(label='Particulate Organic Carbon (mg m^-3)')
    plt.xlabel('Longitude (degrees east)')
    plt.ylabel('Latitude (degrees north)')
    plt.title('MODISA Level-3 Standard Mapped Image for POC')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.savefig('modis_poc_crop.png')
    plt.close()